In [1]:
import pandas as pd

df = pd.read_csv("006_crawling_juznevesti.csv")


In [2]:
def get_metadata(instr: str) -> str:
    instr = instr.replace(u'\xa0', u' ')
    import parse
    try:
        pat = "Gost: {guest}Razgovor vodi{voditi_suffix_and_sep} {host_name} {host_surname}{:s}{body}"
        p = parse.compile(pat)
        results = p.parse(instr).named

    except:
        instr = " ".join(instr.split()).replace(":", "")
        pat = "Gost{guest} Razgovor vodi{voditi_suffix_and_sep} {host_name} {host_surname}{:s}{body}"
        p = parse.compile(pat)
        results = p.parse(instr).named
    finally:
        del results["voditi_suffix_and_sep"]
        return results

num_errors = 0
for i in df.transcript.values:
    try:
        get_metadata(i)
    except:
        num_errors += 1
print(f"Failed for {num_errors} out of {df.shape[0]}")
for j,i in enumerate(df.transcript.values):
    try:
        get_metadata(i)
    except:
        print(j, i.split("\n")[:3])

Failed for 0 out of 299


In [3]:
df["metadata"] = df.transcript.apply(get_metadata)

In [4]:
df = pd.concat([df, df.metadata.apply(pd.Series)], axis=1)

In [5]:
df["host"] = df.apply(lambda row: row["host_name"] + " " + row["host_surname"], axis=1)
df["guest_name"] = df.guest.apply(lambda s: s.split(",")[0])
df["guest_description"] = df.guest.apply(lambda s: ",".join(s.split(",")[1:]))

# Post processing:

Remove leading and trailing 

In [6]:
def remove_excess_whitespace(s:str)->str:
    return " ".join(s.split())
df["host"] = df.host.apply(remove_excess_whitespace)
df["guest_name"] = df.guest_name.apply(remove_excess_whitespace)
df["guest_description"] = df.guest_description.apply(remove_excess_whitespace)
df = df.drop(columns=["host_name", 'host_surname', 'guest', 'metadata'])

Assign `None` to instances where we have no host name:

In [7]:
df.loc[df.host == "None None", "host"] = None

Prepare host questions for host speech tagging:

In [8]:
body = df.body[0]
from typing import List
def extract_questions_from_body(body:str) -> List[str]:
    return [remove_excess_whitespace(i.replace("JV:", "").replace("JV", "")) for i in body.split("\n") if " ".join(i.split()).startswith("JV")]
extract_questions_from_body(body)

['Uprkos pričama da Srbiji ide sve bolje izgleda da građani to ne osećaju na svojoj koži. O tome koliko su pohvale na račun ekonomije u Srbiji realne, razgovaramo sa redovnim profesorom Ekonomskog fakulteta u Nišu Bobanom Stojanovićem. Dobro došli, profesore.',
 'Kada su mediji nedavno preneli informaciju Gradske uprave za privredu da industrijska proizvodnja u Nišu beleži konstantni rast, ova vest je, sudeći po komentarima na društvenim mrežama, nasmejala mnoge građane. Koliko je ta statistika realna?',
 'Kada govorimo i o statistici, i o zapošljavanju, koliko podatak da 64.000 ljudi u Nišu radi, koliko je on dobar, povoljan ili nepovoljan za grad veličine Niša?',
 'Mislite na druge gradove na jugu?',
 'Ko je odgovoran što nema takve politike?',
 'Koje su konkretne mere koje bi unapredile situaciju na jugu?',
 'Šta je rešenje?',
 'Problem je što nema kontrole?',
 'Kakva je situacija što se tiče malih privrednika? Svedoci smo togsa da oni pokrenu neki posao i vrlo brzo odustanu od njeg

In [9]:
df["host_questions"] = df.body.apply(extract_questions_from_body)

In [244]:
df = df[df.yt_hash != "_BzZf0fGg0E"]
df.shape

(298, 10)

In [247]:
pd.set_option("max_colwidth", 30)
df

,url,yt_hash,transcript,exists,path,body,host,guest_name,guest_description,host_questions
0,https://www.juznevesti.com...,00DtyZ2sCUM,"Gost: Boban Stojanović, re...",True,audio/00DtyZ2sCUM.wav,JV: Uprkos pričama da Srbi...,Sanja Janačković,Boban Stojanović,redovni profesor Ekonomsko...,[Uprkos pričama da Srbiji ...
1,https://www.juznevesti.com...,58xZSVbpgkk,"Gost: Slavoljub Vlajković,...",True,audio/58xZSVbpgkk.wav,JV: Nekoliko puta smo pona...,Predrag Blagojević,Slavoljub Vlajković,narodni poslanik SPS iz Niša,[Nekoliko puta smo ponavlj...
2,https://www.juznevesti.com...,YKAqDCyINz4,"Gost: Zoran Žuvković, pred...",True,audio/YKAqDCyINz4.wav,JV: U poslednjih godinu da...,Predrag Blagojević,Zoran Žuvković,predsednik Nove stranke,"[U poslednjih godinu dana,..."
3,https://www.juznevesti.com...,2QShGnPuVjQ,"Gost: Majkl Devenport, Šef...",True,audio/2QShGnPuVjQ.wav,JV: Srbija je napravila ve...,Predrag Blagojević,Majkl Devenport,Šef Delegacije EU u Srbiji,[Srbija je napravila velik...
4,https://www.juznevesti.com...,khEIELaXKwI,"Gost: Jelena Đorđević, Odb...",True,audio/khEIELaXKwI.wav,JV: Zbog toga što nije hte...,Vesna Milić,Jelena Đorđević,Odbornica u Skupštini grad...,[Zbog toga što nije htela ...
...,...,...,...,...,...,...,...,...,...,...
294,https://www.juznevesti.com...,a1cN7-Y29_g,"Gost: Dragan Petković, sav...",True,audio/a1cN7-Y29_g.wav,JV: U četvrtak su u Nišu s...,Predrag Blagojević,Dragan Petković,savetnik za evropske integ...,[U četvrtak su u Nišu sveč...
295,https://www.juznevesti.com...,6CTWDc4amzU,"Gost: Vladica Dimitrov, na...",True,audio/6CTWDc4amzU.wav,JV: Iako je na političkoj ...,Aleksandar Stankov,Vladica Dimitrov,narodni poslanik SNS iz Di...,[Iako je na političkoj sce...
296,https://www.juznevesti.com...,IUIKV9XlWEM,"Gost: Spasoje Milovanović,...",True,audio/IUIKV9XlWEM.wav,JV: Niško Narodno pozorišt...,Milan Zirojević,Spasoje Milovanović,direktor Narodnog pozorišt...,[Niško Narodno pozorište k...
297,https://www.juznevesti.com...,2-F5yoHSn5k,Gost: Vukosava Crnjanski i...,True,audio/2-F5yoHSn5k.wav,JV: Poslednji lokalni izbo...,Predrag Blagojević,Vukosava Crnjanski i Jovan...,,[Poslednji lokalni izbori ...


In [248]:
df.to_json("020_JV_with_metadata.jsonl", orient="records", lines=True)